# Yahoo_live_rl_test Model

In [2]:
import lxml.html as lh
import time
import urllib.request
import argparse


import os
import coloredlogs
from docopt import docopt

## 1. Load model and Define parameters 

In [ ]:
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

## 2. Loading Timezone and Loading Webdriver 

In [ ]:

from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import pytz
tz = pytz.timezone('Asia/Kolkata')


path1 = os.getcwd()
path = path1 + '/chromedriver'

ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(executable_path=path , options=options)

## 3. URL List extraction 

In [ ]:


#This function returns the table of the given url
def Real(url,count):
    if count == 0:
        
        driver.get(url)
        #print(driver)

    else:
        driver.refresh()
        time.sleep(15)

    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

## 4.Evalution Model 

In [ ]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta

        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))

#        done = (t == data_length - 1)
    agent.memory.append((state, action, reward, next_state))

    return total_profit

## 5. Visualization

In [ ]:
#This Function will output a CSV file and this could be used 
#by making certain changes in visualize.ipynb file and visualize the data 
#once the trading is over. 

import pandas as pd
def visualisation(price1,time1):
    visual = pd.DataFrame([[price1[i],time1[i]] for i in range(0,len(price1))], columns=['Time', 'Price'])
    visual.to_csv('visual.csv')
    df = pd.read_csv('visual.csv')

    df = df[['Time', 'Price']]
    df = df.rename(columns={'Price': 'actual', 'Time': 'time'})

    dates = df['time']
    dates = pd.to_datetime(dates, infer_datetime_format=True)
    df['time'] = dates

    print(df.head())
    
    x = time1  
    
    y = price1

  
   

    plt.xlim((min(time1)),(max(time1)))
    plt.xlabel('Time') 
    plt.ylim((min(price1)),(max(price1)))
    plt.ylabel('Price') 


    plt.title('Price vs Time Visualisation of stocks') 

    plt.plot(x, y) 
    plt.draw()
    plt.show() 




## 6. Main Function 

In [ ]:
import datetime
price1 = []
time1=[]
def main():
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = "RELIANCE.NS"
    price =[]
    window_size =10
    time_now = datetime.datetime.now(tz).time()
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        print(count)
        live = Real(url,count)
        count+=1        
        price.append(live)
        price1.append(live)
        time1.append(time_now)
        if count < window_size:
           continue
        model_name='model_debug_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
     
        state = next_state

## Function call 

In [ ]:
if __name__ == "__main__":
   

    coloredlogs.install(level="DEBUG")
    switch_k_backend_device()

    try:
        main()
    except KeyboardInterrupt:
        print("Aborted")
    
        visualisation(price1,time1)
